In [4]:
import pandas as pd
import random
# java = c_class
java = pd.read_csv('./c_class/c_class_mb.txt', sep='\t', names=['user_id','item_id','behavior','timestamp'])
java_user_count = java.groupby('user_id').count().reset_index()[['user_id','item_id']]
java_inter = java[java['user_id'].isin(java_user_count['user_id'])].groupby('user_id').agg('item_id').unique().reset_index()
java_inter['time_avg'] = java.groupby('user_id')['timestamp'].mean().reset_index()['timestamp']
java_inter['time_var'] = java.groupby('user_id')['timestamp'].var().reset_index()['timestamp']
java_inter['time_max'] = java.groupby('user_id')['timestamp'].max().reset_index()['timestamp']
java_inter['time_min'] = java.groupby('user_id')['timestamp'].min().reset_index()['timestamp']
java_inter['inter_len'] = java_inter['item_id'].apply(lambda x : len(x))
java_inter = java_inter[java_inter['inter_len']>=3]
java_inter = java_inter.fillna(0)

from sklearn import preprocessing
from sklearn.cluster import KMeans

k_model = KMeans(n_clusters=4)
train_x = java_inter[['time_avg','time_var','time_max','time_min','inter_len']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
java_inter['scene'] = predict_y

beh_count = java.groupby(['user_id','behavior']).count().reset_index()
java_inter['easy_count'] = beh_count[beh_count['behavior']=='easy']['item_id']
java_inter['medium_count'] = beh_count[beh_count['behavior']=='medium']['item_id']
java_inter['hard_count'] = beh_count[beh_count['behavior']=='hard']['item_id']
java_inter['difficult_count'] = beh_count[beh_count['behavior']=='difficult']['item_id']
java_inter = java_inter.fillna(0)

k_model = KMeans(n_clusters=5)
train_x = java_inter[['easy_count','medium_count','hard_count','difficult_count']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
java_inter['cat'] = predict_y

alldata = []
for user in java_inter['user_id'].tolist():
    # print(user)
    seq_len = len(java_inter[java_inter['user_id']==user]['item_id'].item())
    total_seq = java_inter[java_inter['user_id']==user]['item_id'].item().tolist()
    scene = java_inter[java_inter['user_id']==user]['scene'].item()
    cat = java_inter[java_inter['user_id']==user]['cat'].item()
    i = random.randint(1,seq_len-2)
    data = []
    inter_seq = total_seq[:i]
    pred_path = total_seq[i:seq_len-1]
    if pred_path == []:
        continue
    target = pred_path[-1]
    data.append(user)
    data.append(inter_seq)
    data.append(scene)
    data.append(cat)
    data.append(target)
    data.append(pred_path)
    alldata.append(data)
java_new = pd.DataFrame(alldata,columns = ['user_id','inter_seq','scene','cat','target','pred_path'])
java_new

/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,user_id,inter_seq,scene,cat,target,pred_path
0,1,[57],3,2,60,[60]
1,5,"[56, 55, 57]",3,2,58,[58]
2,7,[55],3,2,56,[56]
3,8,[55],3,2,60,[60]
4,15,[55],3,2,57,[57]
...,...,...,...,...,...,...
1894,4395,[55],1,0,56,[56]
1895,4399,[55],1,2,56,[56]
1896,4405,"[55, 56]",1,2,57,[57]
1897,4440,[55],1,2,57,[57]


In [5]:
java_new[java_new['pred_path'].apply(lambda x: len(x))<=10]

,user_id,inter_seq,scene,cat,target,pred_path
0,1,[57],3,2,60,[60]
1,5,"[56, 55, 57]",3,2,58,[58]
2,7,[55],3,2,56,[56]
3,8,[55],3,2,60,[60]
4,15,[55],3,2,57,[57]
...,...,...,...,...,...,...
1894,4395,[55],1,0,56,[56]
1895,4399,[55],1,2,56,[56]
1896,4405,"[55, 56]",1,2,57,[57]
1897,4440,[55],1,2,57,[57]


In [7]:
java_test = java_new[java_new['pred_path'].apply(lambda x: len(x))<=10].sample(frac=0.3)
java_test

,user_id,inter_seq,scene,cat,target,pred_path
1133,1978,"[56, 55, 57]",2,2,63,"[58, 59, 60, 61, 63]"
356,407,"[56, 55, 57, 58, 59, 60]",0,2,61,[61]
57,91,"[55, 56, 57, 58, 59]",0,2,61,"[60, 61]"
1522,2512,"[56, 57, 58, 59]",2,2,55,"[60, 55]"
1831,3980,[62],1,0,61,[61]
...,...,...,...,...,...,...
1446,2372,"[55, 63, 62, 61, 60]",2,2,57,"[59, 58, 57]"
697,1339,"[55, 56, 57, 58, 59]",0,4,62,"[60, 61, 62]"
1192,2054,"[55, 62, 56, 57]",2,2,60,"[58, 59, 60]"
168,204,[55],0,1,61,"[56, 57, 58, 59, 60, 61]"


In [8]:
java_test.to_csv('c_class/c_class_system.csv',header=True,sep='\t',index=False)

In [16]:
import pandas as pd
# net = computer
net = pd.read_csv('./computer/computer_mb.txt', sep='\t', names=['user_id','item_id','behavior','timestamp'])
net_user_count = net.groupby('user_id').count().reset_index()[['user_id','item_id']]
net_inter = net[net['user_id'].isin(net_user_count['user_id'])].groupby('user_id').agg('item_id').unique().reset_index()

net_inter['time_avg'] = net.groupby('user_id')['timestamp'].mean().reset_index()['timestamp']
net_inter['time_var'] = net.groupby('user_id')['timestamp'].var().reset_index()['timestamp']
net_inter['time_max'] = net.groupby('user_id')['timestamp'].max().reset_index()['timestamp']
net_inter['time_min'] = net.groupby('user_id')['timestamp'].min().reset_index()['timestamp']
net_inter['inter_len'] = net_inter['item_id'].apply(lambda x : len(x))
net_inter = net_inter[net_inter['inter_len']>=3]
net_inter = net_inter.fillna(0)

from sklearn import preprocessing
from sklearn.cluster import KMeans

k_model = KMeans(n_clusters=4)
train_x = net_inter[['time_avg','time_var','time_max','time_min','inter_len']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
net_inter['scene'] = predict_y

beh_count = net.groupby(['user_id','behavior']).count().reset_index()
net_inter['easy_count'] = beh_count[beh_count['behavior']=='easy']['item_id']
net_inter['medium_count'] = beh_count[beh_count['behavior']=='medium']['item_id']
net_inter['hard_count'] = beh_count[beh_count['behavior']=='hard']['item_id']
net_inter['difficult_count'] = beh_count[beh_count['behavior']=='difficult']['item_id']
net_inter = net_inter[net_inter['inter_len']>=3]
net_inter = net_inter.fillna(0)

k_model = KMeans(n_clusters=5)
train_x = net_inter[['easy_count','medium_count','hard_count','difficult_count']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
net_inter['cat'] = predict_y

alldata = []
for user in net_inter['user_id'].tolist():
    # print(user)
    seq_len = len(net_inter[net_inter['user_id']==user]['item_id'].item())
    total_seq = net_inter[net_inter['user_id']==user]['item_id'].item().tolist()
    scene = net_inter[net_inter['user_id']==user]['scene'].item()
    cat = net_inter[net_inter['user_id']==user]['cat'].item()
    i = random.randint(1,seq_len-2)
    data = []
    inter_seq = total_seq[:i]
    pred_path = total_seq[i:seq_len-1]
    if pred_path == []:
        continue
    target = pred_path[-1]
    data.append(user)
    data.append(inter_seq)
    data.append(scene)
    data.append(cat)
    data.append(target)
    data.append(pred_path)
    alldata.append(data)
net_new = pd.DataFrame(alldata,columns = ['user_id','inter_seq','scene','cat','target','pred_path'])
net_new

/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,user_id,inter_seq,scene,cat,target,pred_path
0,1,[122],1,1,132,"[123, 128, 129, 134, 140, 137, 144, 157, 142, ..."
1,2,"[122, 123, 128, 129, 145]",0,3,152,[152]
2,3,"[157, 140]",2,2,163,"[139, 141, 163]"
3,5,"[134, 142, 144, 139]",2,1,151,"[140, 151]"
4,15,"[122, 123]",0,1,128,[128]
...,...,...,...,...,...,...
5689,8278,[144],2,1,132,[132]
5690,8279,"[124, 135, 123, 130, 143, 145, 155, 122, 137, ...",1,0,163,"[132, 149, 160, 131, 163]"
5691,8280,"[134, 128, 122]",2,3,142,[142]
5692,8281,"[122, 128, 142]",2,4,134,"[139, 123, 134]"


In [17]:
net_new[net_new['pred_path'].apply(lambda x: len(x))<=10]

,user_id,inter_seq,scene,cat,target,pred_path
1,2,"[122, 123, 128, 129, 145]",0,3,152,[152]
2,3,"[157, 140]",2,2,163,"[139, 141, 163]"
3,5,"[134, 142, 144, 139]",2,1,151,"[140, 151]"
4,15,"[122, 123]",0,1,128,[128]
6,19,[122],0,1,123,[123]
...,...,...,...,...,...,...
5689,8278,[144],2,1,132,[132]
5690,8279,"[124, 135, 123, 130, 143, 145, 155, 122, 137, ...",1,0,163,"[132, 149, 160, 131, 163]"
5691,8280,"[134, 128, 122]",2,3,142,[142]
5692,8281,"[122, 128, 142]",2,4,134,"[139, 123, 134]"


In [21]:
net_test = net_new[net_new['pred_path'].apply(lambda x: len(x))<=10].sample(frac=0.15)
net_test

,user_id,inter_seq,scene,cat,target,pred_path
561,1045,[122],0,3,124,"[123, 124]"
4897,7272,"[122, 134, 123, 137, 139, 140, 142, 144]",1,1,152,"[151, 138, 128, 163, 148, 149, 152]"
1081,1596,[122],0,3,128,"[123, 128]"
770,1255,[122],0,3,123,[123]
3928,6101,"[122, 123, 128, 163, 142]",2,0,152,"[150, 152]"
...,...,...,...,...,...,...
3726,5892,"[122, 134, 123, 137, 140, 142, 128, 141, 163, ...",1,4,148,"[158, 139, 144, 148]"
1588,2265,[122],0,2,123,[123]
2200,2934,"[122, 123]",0,4,128,"[124, 128]"
2081,2815,[122],0,3,124,"[123, 124]"


In [22]:
net_test.to_csv('computer/computer_system.csv',header=True,sep='\t',index=False)

In [23]:
import pandas as pd
# os = data_science
os = pd.read_csv('./data_sci/data_sci_mb.txt', sep='\t', names=['user_id','item_id','behavior','timestamp'])
os_user_count = os.groupby('user_id').count().reset_index()[['user_id','item_id']]
os_inter = os[os['user_id'].isin(os_user_count['user_id'])].groupby('user_id').agg('item_id').unique().reset_index()

os_inter['time_avg'] = os.groupby('user_id')['timestamp'].mean().reset_index()['timestamp']
os_inter['time_var'] = os.groupby('user_id')['timestamp'].var().reset_index()['timestamp']
os_inter['time_max'] = os.groupby('user_id')['timestamp'].max().reset_index()['timestamp']
os_inter['time_min'] = os.groupby('user_id')['timestamp'].min().reset_index()['timestamp']
os_inter['inter_len'] = os_inter['item_id'].apply(lambda x : len(x))
os_inter = os_inter[os_inter['inter_len']>=3]
os_inter = os_inter.fillna(0)

from sklearn import preprocessing
from sklearn.cluster import KMeans

k_model = KMeans(n_clusters=4)
train_x = os_inter[['time_avg','time_var','time_max','time_min','inter_len']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
os_inter['scene'] = predict_y

beh_count = os.groupby(['user_id','behavior']).count().reset_index()
os_inter['easy_count'] = beh_count[beh_count['behavior']=='easy']['item_id']
os_inter['medium_count'] = beh_count[beh_count['behavior']=='medium']['item_id']
os_inter['hard_count'] = beh_count[beh_count['behavior']=='hard']['item_id']
os_inter['difficult_count'] = beh_count[beh_count['behavior']=='difficult']['item_id']
os_inter = os_inter[os_inter['inter_len']>=3]
os_inter = os_inter.fillna(0)

k_model = KMeans(n_clusters=5)
train_x = os_inter[['easy_count','medium_count','hard_count','difficult_count']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
os_inter['cat'] = predict_y

alldata = []
for user in os_inter['user_id'].tolist():
    # print(user)
    seq_len = len(os_inter[os_inter['user_id']==user]['item_id'].item())
    total_seq = os_inter[os_inter['user_id']==user]['item_id'].item().tolist()
    scene = os_inter[os_inter['user_id']==user]['scene'].item()
    cat = os_inter[os_inter['user_id']==user]['cat'].item()
    i = random.randint(1,seq_len-2)
    data = []
    inter_seq = total_seq[:i]
    pred_path = total_seq[i:seq_len-1]
    if pred_path == []:
        continue
    target = pred_path[-1]
    data.append(user)
    data.append(inter_seq)
    data.append(scene)
    data.append(cat)
    data.append(target)
    data.append(pred_path)
    alldata.append(data)
os_new = pd.DataFrame(alldata,columns = ['user_id','inter_seq','scene','cat','target','pred_path'])
os_new

/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,user_id,inter_seq,scene,cat,target,pred_path
0,2,"[128, 136]",0,1,129,"[125, 123, 135, 127, 124, 132, 129]"
1,4,"[123, 136]",3,1,124,"[125, 124]"
2,5,"[136, 123]",3,0,127,"[125, 124, 126, 127]"
3,7,[123],3,1,124,[124]
4,8,"[125, 136, 123]",3,1,127,"[124, 126, 127]"
...,...,...,...,...,...,...
5033,8733,"[136, 123]",2,4,126,"[124, 126]"
5034,8736,"[123, 124]",1,1,132,"[128, 135, 127, 129, 131, 132]"
5035,8738,[123],2,1,124,[124]
5036,8739,[123],2,1,126,"[124, 126]"


In [24]:
os_new[os_new['pred_path'].apply(lambda x: len(x))<=10]

,user_id,inter_seq,scene,cat,target,pred_path
0,2,"[128, 136]",0,1,129,"[125, 123, 135, 127, 124, 132, 129]"
1,4,"[123, 136]",3,1,124,"[125, 124]"
2,5,"[136, 123]",3,0,127,"[125, 124, 126, 127]"
3,7,[123],3,1,124,[124]
4,8,"[125, 136, 123]",3,1,127,"[124, 126, 127]"
...,...,...,...,...,...,...
5033,8733,"[136, 123]",2,4,126,"[124, 126]"
5034,8736,"[123, 124]",1,1,132,"[128, 135, 127, 129, 131, 132]"
5035,8738,[123],2,1,124,[124]
5036,8739,[123],2,1,126,"[124, 126]"


In [25]:
os_test = os_new[os_new['pred_path'].apply(lambda x: len(x))<=10].sample(frac=0.2)
os_test

,user_id,inter_seq,scene,cat,target,pred_path
1133,1819,"[123, 124]",0,1,129,"[126, 128, 135, 127, 129]"
3594,5964,"[136, 125]",1,1,130,"[127, 135, 124, 123, 126, 128, 131, 130]"
4930,8610,"[124, 123, 128, 135, 127, 129, 131]",1,1,132,[132]
1460,2504,"[136, 123, 125, 126, 124, 131, 129, 128]",3,1,127,"[135, 127]"
1367,2410,"[136, 123, 125, 126, 124]",3,1,128,"[135, 127, 131, 129, 128]"
...,...,...,...,...,...,...
3765,6319,"[129, 128]",2,0,135,[135]
1308,2297,"[136, 125, 123, 126, 124, 129, 128]",3,2,127,[127]
2843,5038,[126],2,2,124,[124]
4249,7083,"[136, 123, 124, 127, 128]",1,2,130,"[135, 126, 129, 131, 130]"


In [27]:
os_test.to_csv('data_sci/data_sci_system.csv',header=True,sep='\t',index=False)